# Introduction

This notebook builds a general GAM model, and then attempts to build a specific GAM model for each of the zip codes. We keep the zip code GAM model if it is better than the general GAM model over all the zip codes. At the end we remove all the zip codes for which we have a better model from the general GAM model, and then we then train the general GAM model with the zip codes for which we have better models excluded.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures, RobustScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_validate, cross_val_predict
from pyearth import Earth
from pygam import LinearGAM, GAM, f, s, te

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import gc; gc.enable()

import warnings
warnings.filterwarnings("ignore")

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Read and Process the Data

Read in the housing data from the CSV file and split it into the training and test sets.

In [3]:
df = pd.read_csv('kc_house_data_clean.csv', index_col=0)
df.head(2)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,waterfront_null,waterfront_ind,yr_renovated_scheme1,yr_renovated_null,yr_renovated_ind
id,,,,,,,,,,,,,,,,,,,,,
7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,1.0,0.0,3,...,98178,47.5112,-122.257,1340,5650,1,0,0,0,0
6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,...,98125,47.7210,-122.319,1690,7639,0,0,2,0,1


In [4]:
X = df[['long',                     #  0
        'lat',                      #  1
        'sqft_living',              #  2
        'grade',                    #  3
        'bathrooms',                #  4
        'condition',                #  5
        'waterfront',               #  6
        'yr_renovated_scheme1',     #  7
        'floors',                   #  8
        'sqft_lot',                 #  9
        'zipcode']].values          # 10
y = df['price'].values

display(X)
display(y)

array([[-1.22257e+02,  4.75112e+01,  1.18000e+03, ...,  1.00000e+00,
         5.65000e+03,  9.81780e+04],
       [-1.22319e+02,  4.77210e+01,  2.57000e+03, ...,  2.00000e+00,
         7.24200e+03,  9.81250e+04],
       [-1.22233e+02,  4.77379e+01,  7.70000e+02, ...,  1.00000e+00,
         1.00000e+04,  9.80280e+04],
       ...,
       [-1.22299e+02,  4.75944e+01,  1.02000e+03, ...,  2.00000e+00,
         1.35000e+03,  9.81440e+04],
       [-1.22069e+02,  4.75345e+01,  1.60000e+03, ...,  2.00000e+00,
         2.38800e+03,  9.80270e+04],
       [-1.22299e+02,  4.75941e+01,  1.02000e+03, ...,  2.00000e+00,
         1.07600e+03,  9.81440e+04]])

array([221900., 538000., 180000., ..., 402101., 400000., 325000.])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Build General Model

Create a general GAMN model for all the zip codes, print a summary, and then print summary statistics.

In [6]:
gam=GAM(n_splines=40, 
        terms=s(2)+s(3)+s(4)+s(5)+s(6)+s(7)+s(8)+s(9)+s(10)+te(0,1)+te(2,9),
        distribution = 'gamma', link='log').fit(X_train,y_train)

In [7]:
gam.summary()

GAM                                                                                                       
=============================================== ==========================================================
Distribution:                         GammaDist Effective DoF:                                    440.4634
Link Function:                          LogLink Log Likelihood:                               -242051.8776
Number of Samples:                        19110 AIC:                                           484986.6819
                                                AICc:                                          485007.6092
                                                GCV:                                                0.0288
                                                Scale:                                              0.0281
                                                Pseudo R-Squared:                                   0.9116
Feature Function                  Lam

In [8]:
y_pred = gam.predict(X_test)

print("R^2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


R^2: 0.8749187619624116
MAE: 65954.74653300311
RMSE: 120143.50873393411


In [9]:
over_or_under = y_test - y_pred
total = len(over_or_under)
over  = len(over_or_under[over_or_under <  0])
under = len(over_or_under[over_or_under >= 0])

print("Total:", total, "\nOver:", over, "\nUnder:", under)

Total: 2124 
Over: 1054 
Under: 1070


# ZIP Codes

The cells below show how many different the number of records per ZIP code. They are ordered from highest count to lowest count.

In [10]:
pd.set_option('display.max_rows', 500)

In [11]:
df.groupby(['zipcode']).size().sort_values(ascending=False)

zipcode
98103    600
98038    586
98115    580
98052    573
98117    553
98042    545
98034    545
98118    505
98006    497
98023    495
98133    492
98059    468
98058    454
98155    445
98074    440
98033    432
98027    409
98125    407
98056    403
98053    402
98001    359
98075    359
98126    353
98092    349
98144    341
98106    335
98116    328
98029    321
98004    316
98199    316
98122    289
98146    287
98008    283
98028    282
98003    280
98040    279
98198    278
98031    273
98072    271
98168    268
98055    268
98112    268
98107    266
98178    262
98136    260
98030    256
98177    254
98166    253
98065    246
98022    231
98105    229
98002    199
98077    198
98011    194
98019    188
98108    186
98119    184
98005    168
98007    141
98188    136
98032    124
98070    117
98014    110
98109    109
98102    104
98010    100
98024     78
98148     57
98039     50
dtype: int64

# Model Dictionary

To store the ZIP code models, we will use a dictionary. The dictionary will have an entry for each ZIP code. Each entry is set to None, to indicate that the default (general) GAM model is better.

In [12]:
model_dict = {}
for zipcode in df.zipcode.unique().tolist():
    model_dict[zipcode] = None

In [13]:
len(model_dict)

69

# Building ZIP Specific Models

The cell below is the work horse and builds for each ZIP code a set of different GAM models to see which one is best by test diffent n_splines values. It then takes the best model for each ZIP code, and compares that against the R^2 value of the general GAM model. If it is better, it will then store the ZIP specific model in the `model_dict` dictionary.

In [15]:
r2_threshold = 0.8671124512126286
better_count = 0
n_splines = 6

for zipcode in df.zipcode.unique().tolist():
    print("processing zip code:", zipcode)
    
    zip_best_r2 = None
    best_nsplines = -1
    
    for n_splines in range(4, 15):
        try:
           # train the model for the zipcode
           train_idx = X_train[:, 10] == zipcode
           zip_gam=GAM(n_splines=n_splines, 
                       terms=s(2)+s(3)+s(4)+s(5)+s(6)+s(7)+s(8)+s(9)+s(10)+te(0,1)+te(2,9),
                       distribution = 'gamma', link='log').fit(X_train[train_idx],y_train[train_idx])
    
           # test the trained model for zip code and check if it is better
           test_idx = X_test[:, 10] == zipcode
           y_pred = zip_gam.predict(X_test[test_idx])
           zip_r2_score = r2_score(y_test[test_idx], y_pred)
        
           if zip_best_r2 is None or zip_r2_score > zip_best_r2:
              best_nsplines = n_splines
              zip_best_r2 = zip_r2_score
        except:
            print("--> failed to train/test model for n_splines=", n_splines)
            continue
    
    print("   successfully trained model with", len(X_train[train_idx]), "rows.")        
    print("   best r2 score for zip code=", zip_best_r2, \
          "r2_threshold=", r2_threshold, \
          "test rows=", len(X_test[test_idx]))
        
    # if the best zip code r2 score is better than threshold save the model
    if zip_best_r2 > r2_threshold:
       train_idx = X_train[:, 10] == zipcode
       zip_gam = GAM(n_splines=best_nsplines, 
                     terms=s(2)+s(3)+s(4)+s(5)+s(6)+s(7)+s(8)+s(9)+s(10)+te(0,1)+te(2,9),
                     distribution = 'gamma', link='log').fit(X_train[train_idx],y_train[train_idx])
       model_dict[zipcode] = zip_gam
       print("*** found better model for zip code", zipcode, "!")
       better_count += 1           

print("Found", better_count, "models !")

processing zip code: 98178
   successfully trained model with 236 rows.
   best r2 score for zip code= 0.793819941970151 r2_threshold= 0.8671124512126286 test rows= 26
processing zip code: 98125
   successfully trained model with 368 rows.
   best r2 score for zip code= 0.7303836856936833 r2_threshold= 0.8671124512126286 test rows= 39
processing zip code: 98028
   successfully trained model with 249 rows.
   best r2 score for zip code= 0.5817469962775501 r2_threshold= 0.8671124512126286 test rows= 33
processing zip code: 98136
   successfully trained model with 235 rows.
   best r2 score for zip code= 0.837302769936436 r2_threshold= 0.8671124512126286 test rows= 25
processing zip code: 98074
   successfully trained model with 391 rows.
   best r2 score for zip code= 0.6630120723555524 r2_threshold= 0.8671124512126286 test rows= 49
processing zip code: 98053
   successfully trained model with 378 rows.
   best r2 score for zip code= 0.9003504481498624 r2_threshold= 0.8671124512126286 te

   successfully trained model with 448 rows.
   best r2 score for zip code= 0.618310792605741 r2_threshold= 0.8671124512126286 test rows= 57
processing zip code: 98199
did not converge
did not converge
   successfully trained model with 276 rows.
   best r2 score for zip code= 0.7739271049957672 r2_threshold= 0.8671124512126286 test rows= 40
processing zip code: 98032
did not converge
   successfully trained model with 116 rows.
   best r2 score for zip code= 0.5045674310154342 r2_threshold= 0.8671124512126286 test rows= 8
processing zip code: 98102
did not converge
did not converge
did not converge
--> failed to train/test model for n_splines= 10
--> failed to train/test model for n_splines= 11
--> failed to train/test model for n_splines= 12
--> failed to train/test model for n_splines= 13
--> failed to train/test model for n_splines= 14
   successfully trained model with 99 rows.
   best r2 score for zip code= 0.9926965463818912 r2_threshold= 0.8671124512126286 test rows= 5
*** foun

# Calculate R^2 with Improved Models

The first cell displays the R^2 performance on the test data set (0.87). The next cell then runs the improved models and stores the result in `y_pred`. After running the improved models, it prints the test performance metrics to compare the results (R^2 = 0.902).

In [16]:
y_pred = gam.predict(X_test)

print("R^2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R^2: 0.8749187619624116
MAE: 65954.74653300311
RMSE: 120143.50873393411


In [17]:
for zipcode in model_dict.keys():
    zip_gam_model = model_dict[zipcode]
    if zip_gam_model is not None:
        print("Improving scores for zipcode", zipcode)
        test_idx = X_test[:, 10] == zipcode
        y_pred[test_idx] = zip_gam_model.predict(X_test[test_idx])

Improving scores for zipcode 98053
Improving scores for zipcode 98115
Improving scores for zipcode 98126
Improving scores for zipcode 98019
Improving scores for zipcode 98092
Improving scores for zipcode 98119
Improving scores for zipcode 98112
Improving scores for zipcode 98052
Improving scores for zipcode 98027
Improving scores for zipcode 98058
Improving scores for zipcode 98056
Improving scores for zipcode 98008
Improving scores for zipcode 98059
Improving scores for zipcode 98144
Improving scores for zipcode 98034
Improving scores for zipcode 98116
Improving scores for zipcode 98102
Improving scores for zipcode 98177
Improving scores for zipcode 98109
Improving scores for zipcode 98033
Improving scores for zipcode 98024
Improving scores for zipcode 98065
Improving scores for zipcode 98039


In [18]:
print("R^2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R^2: 0.9021107354161303
MAE: 61986.37763490695
RMSE: 106284.94654681937


# Refine General Model

By removing the zip codes for which we have better models from the general GAM model, we can reduce the variance further in the general GAM model. The cell below identifies which ZIP coes to remove. It then creates a new set of variables from `X_train` and `y_train`, and removes the zip code entries for those. You can see that it removes around 6.5k entries from the training set (around one third of the data). It then basically rerun the calculations to see what overall improvement we get. The R^2 goes up slightly from 90.2 to 90.5.


In [19]:
zipcodes_to_remove = []

for zipcode in model_dict.keys():
    if model_dict[zipcode] is not None:
        zipcodes_to_remove.append(zipcode)

print(zipcodes_to_remove, len(zipcodes_to_remove))

[98053, 98115, 98126, 98019, 98092, 98119, 98112, 98052, 98027, 98058, 98056, 98008, 98059, 98144, 98034, 98116, 98102, 98177, 98109, 98033, 98024, 98065, 98039] 23


In [20]:
X_train_minus = X_train
y_train_minus = y_train
print("X_train_minus.shape=", X_train_minus.shape, "len(y_train_minus)=", len(y_train_minus))

X_train_minus.shape= (19110, 11) len(y_train_minus)= 19110


In [21]:
for zipcode in zipcodes_to_remove:
    train_idx = X_train_minus[:, 10] == zipcode
    X_train_minus = X_train_minus[~train_idx]
    y_train_minus = y_train_minus[~train_idx]

print("X_train_minus.shape=", X_train_minus.shape, "len(y_train_minus)=", len(y_train_minus))

X_train_minus.shape= (12458, 11) len(y_train_minus)= 12458


In [22]:
gam=GAM(n_splines=40, 
        terms=s(2)+s(3)+s(4)+s(5)+s(6)+s(7)+s(8)+s(9)+s(10)+te(0,1)+te(2,9),
        distribution = 'gamma', link='log').fit(X_train_minus,y_train_minus)

In [23]:
y_pred = gam.predict(X_test)

print("R^2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R^2: 0.8483618613751913
MAE: 73205.91809435622
RMSE: 132284.3439892027


In [24]:
for zipcode in model_dict.keys():
    zip_gam_model = model_dict[zipcode]
    if zip_gam_model is not None:
        print("Improving scores for zipcode", zipcode)
        test_idx = X_test[:, 10] == zipcode
        y_pred[test_idx] = zip_gam_model.predict(X_test[test_idx])

Improving scores for zipcode 98053
Improving scores for zipcode 98115
Improving scores for zipcode 98126
Improving scores for zipcode 98019
Improving scores for zipcode 98092
Improving scores for zipcode 98119
Improving scores for zipcode 98112
Improving scores for zipcode 98052
Improving scores for zipcode 98027
Improving scores for zipcode 98058
Improving scores for zipcode 98056
Improving scores for zipcode 98008
Improving scores for zipcode 98059
Improving scores for zipcode 98144
Improving scores for zipcode 98034
Improving scores for zipcode 98116
Improving scores for zipcode 98102
Improving scores for zipcode 98177
Improving scores for zipcode 98109
Improving scores for zipcode 98033
Improving scores for zipcode 98024
Improving scores for zipcode 98065
Improving scores for zipcode 98039


In [25]:
print("R^2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R^2: 0.9049052318250095
MAE: 61674.330981715175
RMSE: 104756.87574006237


In [26]:
# maximum achieved w/ s(2)+s(3)+s(4)+s(5)+s(6)+s(7)+s(8)+s(9)+s(10)+te(0,1)+te(2,9): .8988
# leave out s(10) for zip code models : 0.8945
# increase splines from 25 to 40 for default model: 0.9049